In [ ]:
!git clone https://github.com/Nomow/02456-deep-learning-image-denoising.git
!pip install -U albumentations
!pip install git+https://github.com/qubvel/segmentation_models.pytorch


Cloning into '02456-deep-learning-image-denoising'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 62 (delta 28), reused 44 (delta 12), pack-reused 0
Unpacking objects: 100% (62/62), done.
     |████████████████████████████████| 102 kB 8.7 MB/s 
     |████████████████████████████████| 47.6 MB 1.4 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-22h_lztr
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-22h_lztr
     |████████████████████████████████| 58 kB 4.1 MB/s 
     |████████████████████████████████| 376 kB 16.1 MB/s 
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorc

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/02456-deep-learning-image-denoising
from pathlib import Path
from dataset import AutoEncoderDataset
from utils import train
from utils import val
import matplotlib.pyplot as plt
import albumentations as albu
import segmentation_models_pytorch as smp
import os
import torch
from torch.utils.data import DataLoader
import albumentations.pytorch
import numpy as np

root_path = Path('/content/gdrive/MyDrive/dermatology_dataset') 
train_path = root_path / 'train/'
val_path = root_path / 'val/'
test_path = root_path / 'test/'
save_path = root_path / "experiment_unetplusplus_no_dropout"
if not os.path.exists(save_path):
    os.makedirs(save_path)



/content/02456-deep-learning-image-denoising


In [ ]:
img_max_size = 512
batch_print = 1
architecture = smp.UnetPlusPlus
encoder = 'timm-regnetx_016'
lr = 0.01
lr_step_size = 70
nb_epochs = 290
batch_size = 15
val_epoch = 5
weight_path = os.path.join("/content/weights", encoder + "-" + architecture.__name__)


net = architecture(encoder_name=encoder,  # UnetPlusPlus
    encoder_weights="imagenet", 
    in_channels=3, 
    classes=3,
    )

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnetx_016-65ca972a.pth" to /root/.cache/torch/hub/checkpoints/regnetx_016-65ca972a.pth


  0%|          | 0.00/35.3M [00:00<?, ?B/s]

In [ ]:
if torch.cuda.is_available():
  net.cuda()
  device = "cuda:0"
else:
  device = "cpu"


In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, lr_step_size, gamma=0.1, last_epoch=-1, verbose=False)

In [ ]:
train_transform = [
    albu.Blur(blur_limit=10, always_apply=False, p=0.6),
    albu.MultiplicativeNoise(multiplier=(0.7, 1.3), p=0.6),
    albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.4, brightness_by_max=True, always_apply=False, p=0.6),
    albu.Downscale(scale_min=0.25, scale_max=0.25, interpolation=0, always_apply=False, p=0.3),
    albu.ISONoise(color_shift=(0.08, 0.15), intensity=(0.2, 0.3), always_apply=False, p=0.6),
    albu.RandomToneCurve(scale=0.3, p=0.6)
]
train_transforms = albu.Compose(train_transform)

preprocess_transform = [
    albu.LongestMaxSize(max_size=img_max_size, always_apply=True),
    albu.PadIfNeeded(min_height=img_max_size, min_width=img_max_size, always_apply=True, border_mode=0),    
    albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    albumentations.pytorch.transforms.ToTensorV2()]

preprocess_transforms = albu.Compose(preprocess_transform)



In [ ]:
train_dataset = AutoEncoderDataset(train_path, preprocess_transforms, train_transforms)
val_dataset = AutoEncoderDataset(val_path, preprocess_transforms, train_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)



In [ ]:
logs = {}
logs["train"] = []
logs["val"] = []
best_loss = np.inf
best_idx = 0
for i in range(nb_epochs):
  print("epoch: ", i + 1)
  net = train(net, criterion, optimizer, scheduler, train_loader, batch_print, device)
  if((i % val_epoch) == 0):
    train_loss = val(net, criterion, train_loader, device)
    tmp_train = {"epoch" : i, "loss" : train_loss}
    logs["train"].append(tmp_train)
    print("train loss: ", train_loss)

    
    val_loss = val(net, criterion, val_loader, device)
    tmp_val = {"epoch" : i, "loss" : val_loss}
    logs["val"].append(tmp_val)
    print("val loss: ", val_loss)
    if(val_loss < best_loss):
      print("best epoch at ", i + 1)
      best_loss = val_loss
      torch.save(net.state_dict(), os.path.join(save_path, "best.pth"))



Streaming output truncated to the last 5000 lines.
[    8] loss: 0.159
[    9] loss: 0.143
[   10] loss: 0.165
[   11] loss: 0.185
[   12] loss: 0.164
[   13] loss: 0.211
[   14] loss: 0.124
[   15] loss: 0.199
[   16] loss: 0.173
[   17] loss: 0.137
[   18] loss: 0.140
[   19] loss: 0.276
epoch:  46
[    1] loss: 0.140
[    2] loss: 0.252
[    3] loss: 0.171
[    4] loss: 0.153
[    5] loss: 0.167
[    6] loss: 0.140
[    7] loss: 0.160
[    8] loss: 0.196
[    9] loss: 0.141
[   10] loss: 0.190
[   11] loss: 0.169
[   12] loss: 0.190
[   13] loss: 0.221
[   14] loss: 0.179
[   15] loss: 0.175
[   16] loss: 0.133
[   17] loss: 0.150
[   18] loss: 0.170
[   19] loss: 0.175
train loss:  0.1518431444761946
val loss:  0.22747992351651192
epoch:  47
[    1] loss: 0.139
[    2] loss: 0.146
[    3] loss: 0.166
[    4] loss: 0.216
[    5] loss: 0.138
[    6] loss: 0.236
[    7] loss: 0.186
[    8] loss: 0.231
[    9] loss: 0.147
[   10] loss: 0.198
[   11] loss: 0.212
[   12] loss: 0.130
[   

In [ ]:
np.save(os.path.join(save_path, "logs.npy"), logs)
